<a href="https://colab.research.google.com/github/szostaklab/aminoacylation/blob/main/gly_structure_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
#IMPORTS
#Install+Import Packages
!pip install mechanize
import random
import re
import mechanize
import numpy as np
from random import *
import matplotlib.pyplot as plt
!pip install Bio
from Bio import SeqIO
from Bio.Seq import Seq
from collections import Counter 
from Bio.Cluster import kcluster
import statistics
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import difflib
import seaborn as sns; sns.set_theme()
from collections import defaultdict
import collections
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
from collections import OrderedDict
import matplotlib.pyplot as plt
#!pip install ipython-autotime

In [116]:
#Upload Data and Define Parameters
aa_sequences = open('/Users/zoeweiss/Desktop/Desktop/lab(s)/szostak/spring 2022/aminoacylation/Str_S1_L001_R1_001.fastq').readlines()
rna_only_sequences = open('/Users/zoeweiss/Desktop/Desktop/lab(s)/szostak/spring 2022/aminoacylation/Str-NaOH_S2_L001_R1_001.fastq').readlines()

#Take the reverse complement? Enter 'yes' or 'no'
rc = 'no'

#Quality Cutoff - what fraction of positions should have <1% error probability?
quality_cutoff = 0.9

#Position motif1 starts at
motif_start = 1
#Motif1 define
motif = 'AAA'
#Position motif2 starts at
motif2_start = 44
#Motif2 define
motif2 = 'AAAAGATCGGAAGAGCACACGTCT'

In [137]:
def preprocess_seqs(list_of_seqs):
    
    quality = list_of_seqs[3::4]
    seqs = list_of_seqs[1::4]

    #Remove duplicates:
    unique_seqs = []
    unique_qualities = []
    indicies_of_unique_seqs = list(np.unique(seqs, return_index=True)[1])
    for index in sorted(np.unique(seqs, return_index = True)[1])[:-1]:
      unique_qualities.append(quality[index])
      unique_seqs.append(seqs[index])

    #Take RC if desired
    pre_trim = []
    if rc == 'yes':
      rc_seqs = []
      for sequence in unique_seqs:
          rc_seqs.append(str(Seq(sequence).reverse_complement()).strip('\n'))
      pre_trim = rc_seqs
    else:
      for sequence in unique_seqs:
          pre_trim.append(str(Seq(sequence)).strip('\n'))
    if rc == 'yes':
      print('Number of unique seqs after rc: '+str(len(pre_trim))) 

    #Trim
    trimmed = []
    for seq in pre_trim:
      trimmed.append(seq[(motif_start-1)+len(motif):(motif2_start-1)])

    print('Number of unique seqs after trimming: '+str(len(trimmed))) 

    #Counts of all seqs
    seq_dict = Counter(trimmed)
    seq_dict_list = [(k, v) for k, v in seq_dict.items()]
    sorted_seq_dict = sorted(seq_dict_list, key = lambda x: x[1], reverse=True)
    sorted_seq_dict_inve = sorted(seq_dict_list, key = lambda x: x[1])

    #Filter by quality
    high_quality = []
    acceptable_quals_1_percent_error = [5,6,7,8,9,':',';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
    for seq in range(len(unique_qualities)):
      overlap=0
      for char in list(set(unique_qualities[seq]).intersection(acceptable_quals_1_percent_error)):
            overlap+=unique_qualities[seq].count(char)
      if overlap/len(unique_qualities[1])>quality_cutoff:
            high_quality.append(trimmed[seq].strip("\n"))
    print('Number of high quality seqs: '+str(len(high_quality))) 
    return sorted_seq_dict, high_quality, seqs


#Function to get secondary structure on RNA Fold
def get_energy_ss(seq):
  url = "http://rna.tbi.univie.ac.at//cgi-bin/RNAWebSuite/RNAfold.cgi"
  br = mechanize.Browser()
  br.set_handle_robots(False) # ignore robots
  br.open(url)
  br.select_form(name="form")
  br["SCREEN"] = seq
  res = br.submit()
  content = res.read()
  energy = int(re.sub("[^0-9]", "", str(content).split('thermodynamic ensemble is')[1].split('kcal/mol')[0]))
  structure = str(content).split('STRUCTURE=')[1].split('target')[0].split('"')[0]
  return energy, structure

#Top high quality seqs
def get_top_seqs(num_to_analyze, num_to_print, ctrl_seqs, high_quality):
    order_dict = {k: v for k, v in sorted(dict(collections.Counter(ctrl_seqs)).items(), key=lambda item: item[1])}
    order_seqs = list(order_dict.keys())
    order_seqs.reverse()
    order_counts = list(order_dict.values())
    order_counts.reverse()

    top_seqs_high_qual = []
    for seq in order_seqs[:num_to_analyze]:
        for high_qual in high_quality:
            if high_qual in seq:
                top_seqs_high_qual.append(seq[seq.index(motif)+len(motif):seq.index(motif2)])
                break
    print('High Quality, Filtered Top Seqs:')
    for i in range(num_to_print):
        print(top_seqs_high_qual[i],order_counts[i])

    return top_seqs


In [136]:
#AA CONTROL
#STATISTICS (note: this takes a few mins per round)
print('Number of seqs in AA: '+str(round(len(aa_sequences)/4)))
aa_correct_seqs = []
for seq in range(len(aa_sequences)):
  if (seq-1)%4==0:
      if aa_sequences[seq][motif_start-1:(motif_start-1)+len(motif)] == motif and aa_sequences[seq][motif2_start-1:(motif2_start-1)+len(motif2)] == motif2:
        aa_correct_seqs+=(aa_sequences[seq-1:seq+3])
print('Number of seqs with correct constant region: '+str(len(aa_correct_seqs[1::4])))


aa_ctrl_sorted_seq_dict, aa_ctrl_high_quality, aa_ctrl_seqs = preprocess_seqs(aa_correct_seqs)
top_seqs_aa_not_rna =  get_top_seqs(100, 10, aa_ctrl_seqs, aa_ctrl_high_quality)



Number of seqs in AA: 3006527
Number of seqs with correct constant region: 1532378
Number of unique seqs after trimming: 1531729
Number of high quality seqs: 1503462
High Quality, Filtered Top Seqs:
TTAGTGAAGGATTTGGCGATCGTTATTACAAAGTAGAGGG 2
CGGCAGTGCGTAGCGGTGGAATGGGGGAGTGGATGGTGGG 2
GCGGTGCAACCAATGGGCTGTTGTGATTGGTGGTAAGTGG 2
CGCTATGTCTGCTGGATTCGGTAGGATAAACGGGTGTAGA 2
ATATGTTTCGCGTGCAACGCCGAGTTGGGCGAGGTAGTAG 2
GGGAATGTTTGAATATGGTATGGGGGATAACGCGGCATGA 2
TAAAGGAGGCAAGCAGCGGAGGGGCGGCGAGCTTTTCGTA 2
AGCTGGGGGTTTATATGTGCGGTCGCAGAGATGAGTTGTT 2
TACGTGCTGCGTGTCCGTGTCGAAGGATGGGCATGTGAGC 2
GATTTGATAGGGATGTAGGGGATCGGCGGGGTTCGGTAAC 2
Low Quality, Unfiltered Top Seqs:
TTAGTGAAGGATTTGGCGATCGTTATTACAAAGTAGAGGG 2
CGGCAGTGCGTAGCGGTGGAATGGGGGAGTGGATGGTGGG 2
GCGGTGCAACCAATGGGCTGTTGTGATTGGTGGTAAGTGG 2
CGCTATGTCTGCTGGATTCGGTAGGATAAACGGGTGTAGA 2
ATATGTTTCGCGTGCAACGCCGAGTTGGGCGAGGTAGTAG 2
GGGAATGTTTGAATATGGTATGGGGGATAACGCGGCATGA 2
TAAAGGAGGCAAGCAGCGGAGGGGCGGCGAGCTTTTCGTA 2
AGCTGGGGGTTTATATGTGCGGTCGCAGAGATGAGT

In [131]:
#RNA CONTROL
#STATISTICS (note: this takes a few mins per round)
print('Number of seqs in RNA: '+str(round(len(rna_only_sequences)/4)))
rna_correct_seqs = []
for seq in range(len(rna_only_sequences)):
  if (seq-1)%4==0:
      if rna_only_sequences[seq][motif_start-1:(motif_start-1)+len(motif)] == motif and rna_only_sequences[seq][motif2_start-1:(motif2_start-1)+len(motif2)] == motif2:
        rna_correct_seqs+=(rna_only_sequences[seq-1:seq+3])
print('Number of seqs with correct constant region: '+str(len(rna_correct_seqs[1::4])))

rna_ctrl_sorted_seq_dict, rna_ctrl_high_quality, rna_ctrl_seqs = preprocess_seqs(rna_correct_seqs)
top_seqs_aa_not_rna =  get_top_seqs(100, 10, rna_ctrl_seqs, rna_ctrl_high_quality)




Number of seqs in RNA: 3449734
Number of seqs with correct constant region: 1171922
Number of unique seqs after trimming: 1171243
Number of high quality seqs: 1149204
GTGTTAAGTTGGGCAGGAGCTCATTTGATCGGAGTTGTAA 2
GGGGGAGAGGTTTGACTGGCTACTGTCGGATTACTGTGGC 2
TGGGCTGGTGCGTTGGAGCCCGAGTACTGCTTTACTGTGT 2
AGAGAAGACTGAGTGCGTGGCATTCGGTGCCTGGTAAAGC 2
AGAAGCCAGAGTTCATCACTTTGCATCTAGCGTAGCGCTC 2
ATTTTGGTGTTTGTACGCGCGGAAAGAATCGGATGTTGTA 2
TGTGCGGGATGTTGCAATTCTGAGCGGCGGTACGGGACGG 2
TTGCACCTAGGTACAGCTTTAGCTGGCGACGGTGAACGTG 2
TGGGATAGCAGCATCTGAGTTCAATGCAGGAAATAAGAAC 2
TCGTTATGCGGCCAAATGCAATTTGGTTTGGGTTAAGCAG 2


In [132]:
#AA NOT RNA 
#STATISTICS (note: this takes a few mins per round)

aa_not_rna_high_quality = list(set(aa_ctrl_high_quality) - set(rna_ctrl_high_quality))
print('Number of seqs in AA but not RNA: '+str(round(len(aa_not_rna_high_quality))))
top_seqs_aa_not_rna =  get_top_seqs(100, 10, aa_ctrl_seqs, aa_not_rna_high_quality)


Number of seqs in AA but not RNA: 1503433
TTAGTGAAGGATTTGGCGATCGTTATTACAAAGTAGAGGG 2
CGGCAGTGCGTAGCGGTGGAATGGGGGAGTGGATGGTGGG 2
GCGGTGCAACCAATGGGCTGTTGTGATTGGTGGTAAGTGG 2
CGCTATGTCTGCTGGATTCGGTAGGATAAACGGGTGTAGA 2
ATATGTTTCGCGTGCAACGCCGAGTTGGGCGAGGTAGTAG 2
GGGAATGTTTGAATATGGTATGGGGGATAACGCGGCATGA 2
TAAAGGAGGCAAGCAGCGGAGGGGCGGCGAGCTTTTCGTA 2
AGCTGGGGGTTTATATGTGCGGTCGCAGAGATGAGTTGTT 2
TACGTGCTGCGTGTCCGTGTCGAAGGATGGGCATGTGAGC 2
GATTTGATAGGGATGTAGGGGATCGGCGGGGTTCGGTAAC 2


In [138]:
#Get the SS of the top sequences
ss_of_top_seqs=[]
for seq in top_seqs_aa_not_rna[:10]:
  if top_seqs_aa_not_rna.index(seq)%2==0:
    print(top_seqs_aa_not_rna.index(seq))
  ss_of_top_seqs.append(get_energy_ss(seq)[1])

0
2
4
6
8


In [139]:
#Cluster by secondary structure
k = 3
matrix = np.asarray([np.fromstring(str(s), dtype=np.uint8) for s in ss_of_top_seqs]);
kmeans = KMeans(init="random", n_clusters=k,n_init=10,max_iter=300,random_state=42)
kmeans.fit(matrix)
labels = kmeans.labels_[:]

all_clusters = []
for cluster_num in range(0, k):
  seqs_in_cluster = []
  for seq in range(len(labels)):
    if labels[seq] == cluster_num:
      seqs_in_cluster.append(ss_of_top_seqs[seq])
  all_clusters.append(seqs_in_cluster)
all_clusters

<ipython-input-139-cbb264d0e58f>:3: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  matrix = np.asarray([np.fromstring(str(s), dtype=np.uint8) for s in ss_of_top_seqs]);


[['.((((((..((((....)))).))))))............',
  '((((...(((((((....))))))).......))))....',
  '.........(((((...(((....)))...))))).....'],
 ['...((.(((...))).))......................',
  '.........((.(((..((((......))))..)))))..',
  '.............((((...((........))...)))).',
  '...................(((((.....)))))......'],
 ['((..(((.((((((..((....)).)))))).))).))..',
  '....((((((..((.((......)).))..))))))....',
  '.....(((((((((((((........)))))))))).)))']]